# ElasticSearch alias and index update testing

For testing out ElasticSearch commands before attaching them to django tasks.
_Needs to be run from the django_app environment_

In [1]:
# Get this working with django 

import os, sys
import django
PROJECTPATH = '.'
sys.path.insert(0, PROJECTPATH)
os.environ.setdefault("DJANGO_SETTINGS_MODULE", "redbox_app.settings")
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
os.chdir(PROJECTPATH)
django.setup()

/Users/rachael.robinson/Library/Caches/pypoetry/virtualenvs/redbox-app-j8Lex4j0-py3.12/lib/python3.12/site-packages/pydantic/_internal/_fields.py:132: UserWarning:

Field "model_name" in LLMCallMetadata has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.



## 0. General ES setup
Getting the settings and the client

In [2]:
from redbox.models import Settings

env = Settings()

es_client = env.elasticsearch_client()

es_root_index = env.elastic_root_index

In [3]:
es_root_index

'redbox-data'

## 1. Adding ES alias

Adds an alias to the existing data.

_Command will be in `django_app.redbox_app.redbox_core.management.commands.add_es_alias.py`_

In [4]:
existing_index = f"{env.elastic_root_index}-chunk"

if not es_client.indices.exists_alias(name=f"{existing_index}-current"):
    es_client.indices.put_alias(index=existing_index, name=f"{existing_index}-current")

2024-09-27 13:18:06,870 INFO _transport: HEAD http://localhost:9200/_alias/redbox-data-chunk-current [status:200 duration:0.025s]


## 2. Reingesting files
This adds a new index for these files.

_Command will be in `django_app/redbox_app/redbox_core/management/commands/reingest_files.py`_

In [4]:
import datetime
import logging

from django_q.tasks import async_task

from redbox.models import Settings
from redbox_app.redbox_core.models import INACTIVE_STATUSES, File
from redbox_app.worker import ingest

In [5]:
default_index = f"{env.elastic_root_index}-chunk"
new_index = f"{default_index}-{datetime.datetime.now(tz=datetime.UTC).strftime('%y%m%d%H%M')}"

for file in File.objects.exclude(status__in=INACTIVE_STATUSES):
    # logger.debug("Reingesting file object %s", file)
    print("Reingesting file object %s", file)
    async_task(
        ingest, file.id, new_index, task_name=file.original_file.name, group="re-ingest"
    )

2024-10-01 10:41:42,812 INFO tasks: Enqueued [redbox_django] 16


Reingesting file object %s frankenstein.txt rachael.robinson@cabinetoffice.gov.uk
Reingesting file object %s ai-sustainability-checklist.pdf rachael.robinson@cabinetoffice.gov.uk


2024-10-01 10:41:42,929 INFO tasks: Enqueued [redbox_django] 17
2024-10-01 10:41:43,050 INFO tasks: Enqueued [redbox_django] 18


Reingesting file object %s i.AI_team_FAQs.docx rachael.robinson@cabinetoffice.gov.uk
Reingesting file object %s i.AI_team_FAQs.docx rachael.robinson@cabinetoffice.gov.uk


2024-10-01 10:41:43,162 INFO tasks: Enqueued [redbox_django] 19
2024-10-01 10:41:43,251 INFO tasks: Enqueued [redbox_django] 20


Reingesting file object %s ai-sustainability-checklist.pdf rachael.robinson@cabinetoffice.gov.uk


## 3. Change which index is aliased

This moves the alias to the newly-created index. It will require finding the index created as one of the args in a reingest-file task in `[domain]/admin/django_q/success/`.

_Command will be in `django_app/redbox_app/redbox_core/management/commands/change_es_aliased_index.py`_

In [6]:
new_index = "redbox-data-chunk-2410011041"
alias = f"{env.elastic_root_index}-chunk-current"

try:
    response = es_client.indices.get_alias(name=alias)
    indices_to_remove = list(response.keys())
except ElasticsearchException as e:
    logger.exception("Error fetching alias", exc_info=e)

# logger.info(f"indices to remove: {indices_to_remove}")
print(f"indices to remove: {indices_to_remove}")
actions = [{"remove": {"index": index, "alias": alias}} for index in indices_to_remove]
actions.append({"add": {"index": new_index, "alias": alias}})
print(actions)

es_client.indices.update_aliases(body={"actions": actions})

2024-10-01 10:42:36,693 INFO _transport: GET http://localhost:9200/_alias/redbox-data-chunk-current [status:200 duration:0.007s]


indices to remove: ['redbox-data-chunk-2409271335']
[{'remove': {'index': 'redbox-data-chunk-2409271335', 'alias': 'redbox-data-chunk-current'}}, {'add': {'index': 'redbox-data-chunk-2410011041', 'alias': 'redbox-data-chunk-current'}}]


2024-10-01 10:42:36,967 INFO _transport: POST http://localhost:9200/_aliases [status:200 duration:0.273s]


ObjectApiResponse({'acknowledged': True})

## 4. Remove old data

This clears out the data from the other indexes we no longer need.

_Command will be in `django_app/redbox_app/redbox_core/management/commands/delete_es_indices.py`_


In [7]:
def list_chunk_indices():
    try:
        # Get all indices
        indices = es_client.cat.indices(format="json")
        # Filter indices that contain '-chunk'
        return [index["index"] for index in indices if "-chunk" in index["index"]]
    except Exception as e:
        # logger.exception("Error fetching indices", exc_info=e)
        print("Error fetching indices")

for index in list_chunk_indices():
    if index != new_index:
        print(f"deleting {index}")
        es_client.indices.delete(index=index)

2024-10-01 10:42:44,455 INFO _transport: GET http://localhost:9200/_cat/indices?format=json [status:200 duration:0.086s]


deleting redbox-data-chunk


2024-10-01 10:42:44,724 INFO _transport: DELETE http://localhost:9200/redbox-data-chunk [status:200 duration:0.268s]


deleting redbox-data-chunk-2409271335


2024-10-01 10:42:44,986 INFO _transport: DELETE http://localhost:9200/redbox-data-chunk-2409271335 [status:200 duration:0.261s]


deleting redbox-data-integration-chunk


2024-10-01 10:42:45,296 INFO _transport: DELETE http://localhost:9200/redbox-data-integration-chunk [status:200 duration:0.309s]
